# Intro

This is a demo of the paper Teachable Reinforcement Learning via Advice Distillation showing how humans can use advice to coach agents through new tasks.

For more details, check out our NeurIPS paper and video: https://neurips.cc/Conferences/2021/Schedule?showEvent=27834

# Setup - do this once

To avoid version conflicts, we recommend running this in a conda env with python 3.7.

    conda create --name teachable_rl python=3.7
    conda activate teachable_rl
    pip install notebook
    
You either need to run this on a device with a display. If you're running on a machine without one, use port forwarding:

    ssh -L 9999:localhost:9999 INSERT_SERVER_NAME
    
    jupyter notebook --no-browser --port 9999


In [1]:
!git clone https://github.com/aliengirlliv/teachable 1> /dev/null

fatal: destination path 'teachable' already exists and is not an empty directory.


In [2]:
cd teachable

/home/olivia/Documents/Teachable/demo/teachable


In [3]:
!git checkout sac 1> /dev/null
!pip install -r reqs.txt 1> /dev/null

Already on 'sac'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.8.2+cu101 requires torch==1.7.1, but you have torch 1.8.1 which is incompatible.
You should consider upgrading via the '/home/olivia/anaconda3/envs/teachable/bin/python -m pip install --upgrade pip' command.


In [4]:
cd ..

/home/olivia/Documents/Teachable/demo


# Setup - Do this each time you reload the notebook

In [5]:
%matplotlib tk

In [6]:
cd teachable

/home/olivia/Documents/Teachable/demo/teachable


In [7]:
from final_demo import *
import ipywidgets as widgets
from IPython.display import HTML

In [8]:
cd teachable

[Errno 2] No such file or directory: 'teachable'
/home/olivia/Documents/Teachable/demo/teachable


# Setup

## Instructions
1. Select the collection mode.
    - "Advice" runs the "Improvement" phase of our method, allowing you to coach an agent through
    - "Demos" lets you collect trajectories providing actions each timestep.
2. Select a save name (any string describing this experiment).
3. Collect demos below


In [9]:
widgets.Box(items)

Box(children=(ToggleButtons(description='Collect Mode', options=('Advice', 'Demos'), value='Advice'), Text(val…

# Collection
To collect data, run the block below. A window will open which lets you collect data.  

In our human exps, we found you can reach good performance on this env with about 30 mins of human collection time.

## Task
The agent's task is to unlock a door by collecting a matching-colored key and using it on the corresponding door. (To speed up training time, we always spawn the agent in the same room as the locked door.)

### Using Advice

The agent you will be coaching has been pretrained to understand Waypoint advice. It has never seen this particular environment/task before, and has never had to unlock a door. Click on a square to tell the agent to head there and manipulate any item present. Use the scrollbar to advance.

### Providing Demos
Use the arrow keys to navigate, Page Up/Down to manipulate objects, and Space to open doors.


In [12]:
collector = HumanFeedback(collect_type=env_params.collect_type, 
                          save_path=env_params.save_path, seed=123)

# Train

Here, we train an advice-free policy on the collected trajectories using the buffer of collected trajectories.


It will train for 20 itrs, but feel free to pause before then.

In [13]:
args = make_args(collector, env_params.save_path)  
run_experiment(args)

Logging to /home/olivia/Documents/Teachable/demo/teachable/logs/yolo
loaded buffer /home/olivia/Documents/Teachable/demo/teachable/logs/yolo_buffer/buffer/train_buffer.pkl 0 0
Done on timestep 60 0
Done on timestep 60 0


KeyboardInterrupt: 

# Visualize

Play video saved during training. This agent does not receive advice.

In [ ]:
html_str = display_trained_model(env_params.save_path)
HTML(html_str)